In [14]:
#r "nuget: Microsoft.SemanticKernel, *-*"

Installed Packages Microsoft.SemanticKernel, 0.13.277.1-preview

In [15]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.TemplateEngine;
using System.Collections.Generic;
using System.Threading.Tasks;

In [16]:
var kernel = KernelBuilder.Create();

In [17]:
kernel.Config.AddAzureTextCompletionService(
    "davinci",      
    "GPT3Model",                  
    "Your Endpoint",    
    "Your Key"       
);

In [18]:
string skPrompt = """

请帮我把 {{$input}} 进行分类，类型包括天气，课程，生成式，如果不清楚，请回答不确认，参考如下：

问: 会下雨吗？ 类别:天气
问: 今天温度？ 类别:天气
问: 适度多少？ 类别:天气
问: 什么是新能源车？ 类别: 课程
问: 电动车的特点 类别: 课程
问: 概念是什么？ 类别: 课程
问: 课程相关的内容有哪些？ 类别: 课程
问: 写一首诗歌？ 类别: 生成式
问: 翻译一下 类别: 生成式
问: 计算结果 类别: 生成式

天气输出 1 ， 课程输出 2 ， 生成式输出 3 ，不确认输出 0，并参考以下格式输出

{""question"":""{{$input}}"",""label"":""{{$label}}""}

""";

In [19]:
var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
      MaxTokens = 60,
      FrequencyPenalty = (float)0,
      PresencePenalty = (float)0
    }
};

In [20]:
var promptTemplate = new PromptTemplate(
    skPrompt,                        // Prompt template defined in natural language
    promptConfig,                    // Prompt configuration
    kernel                           // SK instance
);

In [21]:
skPrompt


请帮我把 {{$input}} 进行分类，类型包括天气，课程，生成式，如果不清楚，请回答不确认，参考如下：

问: 会下雨吗？ 类别:天气
问: 今天温度？ 类别:天气
问: 适度多少？ 类别:天气
问: 什么是新能源车？ 类别: 课程
问: 电动车的特点 类别: 课程
问: 概念是什么？ 类别: 课程
问: 课程相关的内容有哪些？ 类别: 课程
问: 写一首诗歌？ 类别: 生成式
问: 翻译一下 类别: 生成式
问: 计算结果 类别: 生成式

天气输出 1 ， 课程输出 2 ， 生成式输出 3 ，不确认输出 0，并参考以下格式输出

{""question"":""{{$input}}"",""label"":""{{$label}}""}


In [22]:

var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);

In [23]:
var classificationFunction = kernel.RegisterSemanticFunction("QASkill", "QuestionClassification", functionConfig);

In [24]:
classificationFunction


Microsoft.SemanticKernel.Orchestration.SKFunction Name QuestionClassification SkillName QASkill Description IsSemantic True RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 60 StopSequences Parameters index value 0 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name input Description DefaultValue 1 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name label Description DefaultValue

In [25]:
var input = "今天广州天气怎么样？";

In [26]:
var classification = await kernel.RunAsync(input, classificationFunction);

Console.WriteLine(classification);


答案：{"question":"今天广州天气怎么样？","label":"1"}
